In [3]:
import pandas as pd

# The Co-working Network

In [86]:
import numpy
import networkx
import itertools
from collections import Counter

class CoworkingNetwork(networkx.Graph):
    """
    Class for coworking networks. Extends networkx Graph class.
    
    Parameters
    ----------
    namesSets : iterable
        An iterable of iterables containing names used to compose cliques 
        in the network.

    weighted : bool
        If set to True the resulting network will have weighted edges. Default False.
        
    namesMap : NamesMap
        A NamesMap object for normalizing nodes names.
        
    Examples
    --------
    >>> namesSets = [ ['a','b','c'], ['d','e'], ['a','c'] ]
    >>> CoworkingNetwork( namesSets, weighted=True).edges(data=True)
    [('b', 'a', {'weight': 1}),
     ('b', 'c', {'weight': 1}),
     ('a', 'c', {'weight': 2}),
     ('e', 'd', {'weight': 1})]
    
    >>> CoworkingNetwork( namesSets ).edges(data=True)
    [('b', 'a', {}), 
     ('b', 'c', {}), 
     ('a', 'c', {}), 
     ('e', 'd', {})]
    """
    def __init__(self, data=None, namesSets=None, weighted=False, namesMap=None, **attr):
       
        if namesSets is not None:
            if namesMap:
                nmap = namesMap.getMap()
                namesSets = [ [ nmap[n] for n in nset ] for nset in namesSets ]

            cliques = map( lambda n: itertools.combinations(n,r=2), namesSets )
            data = [ e for edges in cliques for e in edges ]

        super().__init__(data=data,**attr)

        if weighted:
            edges = data
            edges_weights = Counter(edges)

            for (u,v),w in edges_weights.items():
                try:
                    self[u][v]['weight'] += w
                except:
                    self[u][v]['weight'] = w


class SpeciesCollectorsNetwork(networkx.Graph):
    """
    Class for Species-collectors networks
    
    Parameters
    ----------
    
    A dataframe with two columns: an atomized collectors names 
    """
    def __init__(self, data=None, species=None, collectorsNames=None, weighted=False, namesMap=None, **attr):
        
        self._speciesBag_matrix = None
        
        set_bipartite_attr=False
        if species is not None and collectorsNames is not None:
            if namesMap:
                nmap = namesMap.getMap()
                collectorsNames = [ [ nmap[n] for n in nset ] for nset in collectorsNames ]
            
            # build edges
            if len(species)==len(collectorsNames):
                species = list(species)
                collectorsNames = list(collectorsNames)
                
                data = [ (sp,col) for i,sp in enumerate(species) for col in collectorsNames[i] ]
                set_bipartite_attr=True

        super().__init__(data=data,**attr)
        
        if set_bipartite_attr:
            networkx.set_node_attributes( self, 'bipartite', dict( (n,1) for n in species) )
            networkx.set_node_attributes( self, 'bipartite', dict( (n,0) for cols in collectorsNames for n in cols) )
            
        if weighted:
            edges = data
            edges_weights = Counter(edges)

            for (u,v),w in edges_weights.items():
                try:
                    self[u][v]['weight'] += w
                except:
                    self[u][v]['weight'] = w    
    
    def _buildSpeciesBagMatrix( self, col_sp_order=None ):
        col_sp_order=(sorted(self.getCollectorsNodes()),sorted(self.getSpeciesNodes())) if col_sp_order is None else col_sp_order
        m = networkx.bipartite.biadjacency_matrix(self,row_order=col_sp_order[0],column_order=col_sp_order[1])
        self._speciesBag_matrix = (*col_sp_order,m)
                    
    def getSpeciesNodes(self,data=False):
        return [ (n,d) if data==True else n for n,d in self.nodes(data=True) if d['bipartite']==1 ]
        
    def getCollectorsNodes(self,data=False):
        return [ (n,d) if data==True else n for n,d in self.nodes(data=True) if d['bipartite']==0 ]
    
    def getSpeciesBag( self, collectorName ):
        """
        Parameters
        ----------
        
        Returns
        -------
        A tuple (spIds, vector), where the first element is a list containing all species names and
        the second is the vector containing their counts.
        """
        if self._speciesBag_matrix is None:
            self._buildSpeciesBagMatrix()
            
        colList, spList, m = self._speciesBag_matrix
        i = colList.index(collectorName)
        vector = m.getrow(i)
        return (spList, vector)

In [87]:
import matplotlib.pyplot as plt

plt.plot()

In [39]:
dsetpath = "/home/pedro/datasets/ub_herbarium/occurrence.txt"
occs = pd.read_table(dsetpath, usecols=['recordedBy','scientificName']).dropna()
occs['scientificName'] = occs['scientificName'].astype(str)
occs['recordedBy'] = occs['recordedBy'].astype(str)

In [40]:
from modules.cleaning.names import read_NamesMap_fromJson, NamesAtomizer
from modules.cleaning.names import namesFromString
nm = read_NamesMap_fromJson('./ub_names_map_2.json')

atomizingOp = lambda x: namesFromString(x)
na = NamesAtomizer(atomizingOp)
occs['recordedBy_atomized'] = na.atomize(occs['recordedBy'])

/home/pedro/datascience_biodiversity/notebooks/modules/cleaning/names.py:536: UserWarning: A names map was created without a normalization function!
  warn("A names map was created without a normalization function!")


In [41]:
cwn = CoworkingNetwork(namesSets=occs['recordedBy_atomized'],weighted=True, namesMap=nm)

In [78]:
scn = SpeciesCollectorsNetwork(species=occs['scientificName'], collectorsNames=occs['recordedBy_atomized'],
                               weighted=True, namesMap=nm)

In [43]:
scn.edges(data=True)

[('Annona monticola Mart.', 'irwin,hs', {'weight': 19}),
 ('Annona monticola Mart.', 'grearjunior,jw', {'weight': 2}),
 ('Annona monticola Mart.', 'souza,rr', {'weight': 5}),
 ('Annona monticola Mart.', 'santos,rrb', {'weight': 6}),
 ('Annona monticola Mart.', 'anderson,wr', {'weight': 5}),
 ('Annona monticola Mart.', 'onishi,e', {'weight': 2}),
 ('Annona monticola Mart.', 'fonseca,sf', {'weight': 5}),
 ('Annona monticola Mart.', 'ramos,pcm', {'weight': 1}),
 ('Annona monticola Mart.', 'munhoz,cbr', {'weight': 1}),
 ('Annona monticola Mart.', 'kirkbride-junior,jh', {'weight': 2}),
 ('Annona monticola Mart.', 'kallin-arroyo,mt', {'weight': 1}),
 ('Annona monticola Mart.', 'hill,sr', {'weight': 1}),
 ('Annona monticola Mart.', 'fonsecafilho,j', {'weight': 2}),
 ('Annona monticola Mart.', 'ratter,ja', {'weight': 4}),
 ('Annona monticola Mart.', 'bridgewater,s', {'weight': 1}),
 ('Annona monticola Mart.', 'soares-silva,lh', {'weight': 2}),
 ('Annona monticola Mart.', 'duval,fg', {'weight':

In [44]:
import networkx as nx
list(nx.connected_component_subgraphs(cwn))[5].nodes()
list(nx.connected_component_subgraphs(scn))[5].nodes()

['souza,mfg', 'Calathea grazielae H.Kenn. & Marcelo', 'kennedy,h']

In [72]:
del(scn._speciesBag_matrix)

In [83]:
scn.getSpeciesBag('proenca,ceb').todense()

matrix([[0, 0, 0, ..., 0, 0, 0]])